# **DEPENDENCIES**

---

```json
{
    "depends-on": [
        "src/DataFactory/Utils/Functions.ipynb",
        "src/DataFactory/Utils/Schemas.ipynb",
        "src/DataFactory/Utils/Protos.ipynb",
        "src/utils/conversions.ipynb",
        "src/utils/schemas.ipynb",
        "src/utils/protos.ipynb"
    ]
}
```

## **STATIC FILE**

Here we create a table for the "static part" of the Case National file: the numbers of Covid-19 infected people per day.

In [ ]:
IF NOT EXISTS (SELECT * FROM sys.tables WHERE [object_id] = OBJECT_ID('[VWSINTER].[RIVM_COVID_19_CASE_NATIONAL]'))
CREATE TABLE [VWSINTER].[RIVM_COVID_19_CASE_NATIONAL](
	[ID] [bigint] IDENTITY(1,1) NOT NULL,
	[DATE_FILE] [datetime] NULL,
	[DATE_STATISTICS] [datetime] NULL,
	[DATE_STATISTICS_TYPE] [varchar](100) NULL,
	[AGEGROUP] [varchar](100) NULL,
	[SEX] [varchar](100) NULL,
	[PROVINCE] [varchar](100) NULL,
	[HOSPITAL_ADMISSION] [varchar](100) NULL,
	[DECEASED] [varchar](100) NULL,
	[WEEK_OF_DEATH] [varchar](100) NULL,
	[MUNICIPAL_HEALTH_SERVICE] [varchar](100) NULL,
	[DATE_LAST_INSERTED] [datetime] DEFAULT GETDATE()
) ON [PRIMARY]
GO


In [ ]:
IF NOT EXISTS (SELECT * FROM sys.tables WHERE [object_id] = OBJECT_ID('[VWSSTATIC].[RIVM_COVID_19_CASE_NATIONAL]'))
CREATE TABLE [VWSSTATIC].[RIVM_COVID_19_CASE_NATIONAL](
	[ID] [int] IDENTITY(1,1) NOT NULL,
	[DATE_FILE] [varchar](100) NULL,
	[DATE_STATISTICS] [varchar](100) NULL,
	[DATE_STATISTICS_TYPE] [varchar](100) NULL,
	[AGEGROUP] [varchar](100) NULL,
	[SEX] [varchar](100) NULL,
	[PROVINCE] [varchar](100) NULL,
	[HOSPITAL_ADMISSION] [varchar](100) NULL,
	[DECEASED] [varchar](100) NULL,
	[WEEK_OF_DEATH] [varchar](100) NULL,
	[MUNICIPAL_HEALTH_SERVICE] [varchar](100) NULL,
	[DATE_LAST_INSERTED] [datetime] DEFAULT GETDATE(),
	[VERSION] [varchar](50) NULL
) ON [PRIMARY]
GO

We use the `@cut_off_date` variable to determine wich data we preserve in the "static table". The data from the last inserted file will be used and from this data we use the data with a `[DATE_STATISTICS]` that lies before the `@cut_off_date`.

In [ ]:
/* THIS HAS ALREADY BEEN RUN AND SHOULD NOT RUN AGAIN
DECLARE @cut_off_date DATE = CAST('2021-10-03' AS DATE)

-- TRUNCATE TABLE [VWSSTATIC].[RIVM_COVID_19_CASE_NATIONAL]

-- SET IDENTITY_INSERT [VWSSTATIC].[RIVM_COVID_19_CASE_NATIONAL] ON

INSERT INTO [VWSSTATIC].[RIVM_COVID_19_CASE_NATIONAL] (
	[DATE_FILE],
	[DATE_STATISTICS],
	[DATE_STATISTICS_TYPE],
	[AGEGROUP],
	[SEX],
	[PROVINCE],
	[HOSPITAL_ADMISSION],
	[DECEASED],
	[WEEK_OF_DEATH],
	[MUNICIPAL_HEALTH_SERVICE]
)
SELECT
	[DATE_FILE],
	[DATE_STATISTICS],
	[DATE_STATISTICS_TYPE],
	[AGEGROUP],
	[SEX],
	[PROVINCE],
	[HOSPITAL_ADMISSION],
	[DECEASED],
	[WEEK_OF_DEATH],
	[MUNICIPAL_HEALTH_SERVICE]
FROM 
    [VWSINTER].[RIVM_COVID_19_CASE_NATIONAL] WITH(NOLOCK)
WHERE [DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSINTER].[RIVM_COVID_19_CASE_NATIONAL] WITH(NOLOCK))
    AND CAST([dbo].[TRY_CONVERT_TO_DATETIME]([DATE_STATISTICS]) AS DATE) <= @cut_off_date

-- SET IDENTITY_INSERT [VWSSTATIC].[RIVM_COVID_19_CASE_NATIONAL] OFF
*/

Now we Alter the procedure that copies data from the VWSTAGE schema to the VWSINTER schema, to combine data from both the newly created (and populated) table in VWSSTATIC and the VWSSTAGE-table.

In [ ]:
-- 1) CREATE STORED PROCEDURE(S).....
CREATE OR ALTER PROCEDURE [dbo].[SP_RIVM_COVID_19_NATIONAL_INTER]
 AS
 BEGIN
     -- WITH CTE As (
     --     SELECT
     --         [DATE_FILE],
     --         [DATE_STATISTICS],
     --         [DATE_STATISTICS_TYPE],
     --         [AGEGROUP],
     --         [SEX],
     --         [PROVINCE],
     --         [HOSPITAL_ADMISSION],
     --         [DECEASED],
     --         [WEEK_OF_DEATH],
     --         [MUNICIPAL_HEALTH_SERVICE]
     --     FROM
     --         [VWSSTATIC].[RIVM_COVID_19_CASE_NATIONAL]
     --     WHERE [DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSSTATIC].[RIVM_COVID_19_CASE_NATIONAL])
     -- )
     INSERT INTO [VWSINTER].[RIVM_COVID_19_CASE_NATIONAL] (
         [DATE_FILE],
         [DATE_STATISTICS],
         [DATE_STATISTICS_TYPE],
         [AGEGROUP],
         [SEX],
         [PROVINCE],
         [HOSPITAL_ADMISSION],
         [DECEASED],
         [WEEK_OF_DEATH],
         [MUNICIPAL_HEALTH_SERVICE]
     )
     SELECT
         [dbo].[TRY_CONVERT_TO_DATETIME]([DATE_FILE]),
         [dbo].[TRY_CONVERT_TO_DATETIME]([DATE_STATISTICS]),
         TRIM([DATE_STATISTICS_TYPE]),
         TRIM([AGEGROUP]),
         TRIM([SEX]),
         TRIM([PROVINCE]),
         TRIM([HOSPITAL_ADMISSION]),
         TRIM([DECEASED]),
         TRIM([WEEK_OF_DEATH]),
         TRIM([MUNICIPAL_HEALTH_SERVICE])
     FROM
         [VWSSTAGE].[RIVM_COVID_19_CASE_NATIONAL]
     WHERE [DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSSTAGE].[RIVM_COVID_19_CASE_NATIONAL])
     -- UNION ALL
     -- SELECT
     --     [dbo].[TRY_CONVERT_TO_DATETIME]([DATE_FILE]),
     --     [dbo].[TRY_CONVERT_TO_DATETIME]([DATE_STATISTICS]),
     --     TRIM([DATE_STATISTICS_TYPE]),
     --     TRIM([AGEGROUP]),
     --     TRIM([SEX]),
     --     TRIM([PROVINCE]),
     --     TRIM([HOSPITAL_ADMISSION]),
     --     TRIM([DECEASED]),
     --     TRIM([WEEK_OF_DEATH]),
     --     TRIM([MUNICIPAL_HEALTH_SERVICE])
     -- FROM CTE
 END;
GO




In [ ]:
-- 1) CREATE STORED PROCEDURE(S).....
CREATE OR ALTER PROCEDURE [dbo].[SP_RIVM_COVID_19_NATIONAL_STATIC_TO_INTER] (
     @date_of_insertion DATETIME = GETDATE
 )
 AS
 BEGIN
     INSERT INTO [VWSINTER].[RIVM_COVID_19_CASE_NATIONAL] (
         [DATE_FILE],
         [DATE_STATISTICS],
         [DATE_STATISTICS_TYPE],
         [AGEGROUP],
         [SEX],
         [PROVINCE],
         [HOSPITAL_ADMISSION],
         [DECEASED],
         [WEEK_OF_DEATH],
         [MUNICIPAL_HEALTH_SERVICE],
         [DATE_LAST_INSERTED]
     )
     SELECT
         [dbo].[TRY_CONVERT_TO_DATETIME]([DATE_FILE]),
         [dbo].[TRY_CONVERT_TO_DATETIME]([DATE_STATISTICS]),
         TRIM([DATE_STATISTICS_TYPE]),
         TRIM([AGEGROUP]),
         TRIM([SEX]),
         TRIM([PROVINCE]),
         TRIM([HOSPITAL_ADMISSION]),
         TRIM([DECEASED]),
         TRIM([WEEK_OF_DEATH]),
         TRIM([MUNICIPAL_HEALTH_SERVICE]),
         @date_of_insertion
     FROM
         [VWSSTATIC].[RIVM_COVID_19_CASE_NATIONAL] WITH(NOLOCK)
     WHERE [DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSSTATIC].[RIVM_COVID_19_CASE_NATIONAL] WITH(NOLOCK))
 END;
GO




## **TABLES**

### INTER

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATION FOR MORE INFORMATION.

SET ANSI_NULLS ON
GO

SET QUOTED_IDENTIFIER ON
GO

-- 1) CREATE TABLE(S).....
IF NOT EXISTS(SELECT * FROM SYS.TABLES WHERE [OBJECT_ID] = OBJECT_ID(N'[VWSINTER].[RIVM_COVID_19_CASE_NATIONAL]'))
BEGIN
    CREATE TABLE [VWSINTER].[RIVM_COVID_19_CASE_NATIONAL](
        [ID] [bigint] IDENTITY(1,1) NOT NULL,
        [DATE_FILE] [datetime] NULL,
        [DATE_STATISTICS] [datetime] NULL,
        [DATE_STATISTICS_TYPE] [varchar](100) NULL,
        [AGEGROUP] [varchar](100) NULL,
        [SEX] [varchar](100) NULL,
        [PROVINCE] [varchar](100) NULL,
        [HOSPITAL_ADMISSION] [varchar](100) NULL,
        [DECEASED] [varchar](100) NULL,
        [WEEK_OF_DEATH] [varchar](100) NULL,
        [MUNICIPAL_HEALTH_SERVICE] [varchar](100) NULL,
        [DATE_LAST_INSERTED] [datetime] DEFAULT GETDATE()
    );
END


### DEST

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATION FOR MORE INFORMATION.

SET ANSI_NULLS ON
GO

SET QUOTED_IDENTIFIER ON
GO

-- 1) CREATE TABLE(S).....
IF NOT EXISTS(SELECT * FROM SYS.TABLES WHERE [OBJECT_ID] = OBJECT_ID(N'[VWSDEST].[POSITIVE_TESTED_PEOPLE_PER_AGE_GROUP_OVER_TIME]'))
BEGIN
    CREATE TABLE [VWSDEST].[POSITIVE_TESTED_PEOPLE_PER_AGE_GROUP_OVER_TIME](
	[ID] [int] PRIMARY KEY IDENTITY(1,1),
	[AGEGROUP] [varchar](100) NULL,
	[DATE] [datetime] NULL,
	[DATE_UNIX] [bigint] NULL,
	[CASES_7D_MOVING_AVERAGE] [decimal](38, 6) NULL,
	[CASES_7D_MOVING_AVERAGE_100K] [numeric](38, 6) NULL,
	[CASES] [int] NULL,
	[DATE_LAST_INSERTED] [datetime] DEFAULT GETDATE()
    )
END


## **VIEWS**

In [ ]:
-- -- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN   VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- -- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATIONFOR MORE INFORMATION.

--1) CREATE VIEW(S).....
CREATE OR ALTER VIEW [VWSDEST].[V_POSITIVE_TESTED_PEOPLE_70_PLUS_OVER_TIME]
AS
	SELECT   
			SUM([CASES]) AS [INFECTED_AGE_70_PLUS],
			[DATE_UNIX] AS [DATE_UNIX],
			[dbo].[CONVERT_DATETIME_TO_UNIX]([DATE_LAST_INSERTED]) AS [DATE_OF_INSERTION_UNIX]
	FROM [VWSDEST].[POSITIVE_TESTED_PEOPLE_PER_AGE_GROUP_OVER_TIME]
	WHERE [AGEGROUP] IN ('70-79', '80-89', '90+')
	AND [DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSDEST].[POSITIVE_TESTED_PEOPLE_PER_AGE_GROUP_OVER_TIME])
	GROUP BY [DATE_UNIX], [DATE_LAST_INSERTED]
GO




In [ ]:
-- -- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN   VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- -- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATIONFOR MORE INFORMATION.

--1) CREATE VIEW(S).....
CREATE OR ALTER VIEW [VWSDEST].[V_DIFFERENCES_POSITIVE_TESTED_PEOPLE_70_PLUS_OVER_TIME] AS 

  WITH BASE_CTE AS 
(
    SELECT
		[DATE_UNIX],
		LAG([DATE_UNIX], 1) OVER (ORDER BY [DATE_UNIX]) AS [OLD_DATE_UNIX],
		[INFECTED_AGE_70_PLUS] AS [NEW_VALUE],
		LAG([INFECTED_AGE_70_PLUS], 1) OVER (ORDER BY [DATE_UNIX]) AS [OLD_VALUE],
		ROW_NUMBER() OVER (ORDER BY [DATE_UNIX] DESC) AS RANK_DATE
    FROM [VWSDEST].[V_POSITIVE_TESTED_PEOPLE_70_PLUS_OVER_TIME]
)
SELECT
    [DATE_UNIX] AS [NEW_DATE_UNIX],
    [OLD_DATE_UNIX] AS [OLD_DATE_UNIX],
    [OLD_VALUE],
	ROUND([NEW_VALUE] - [OLD_VALUE], 1) AS [DIFFERENCE]
FROM BASE_CTE
  WHERE RANK_DATE = 1 --Only the most recent record
GO




## **PROTOS**

Basic view

In [ ]:
-- 1) SET ENVIRONMENTAL VARIABLES.....
DECLARE @view_name VARCHAR(256) = 'VWSDEST.V_POSITIVE_TESTED_PEOPLE_70_PLUS_OVER_TIME',
        @view_description VARCHAR(256),
        @item_name VARCHAR(256) = 'vulnerable_tested_per_age_group',
        @config_description VARCHAR(256),
        @constraint_value VARCHAR(50) = NULL,
        @constraint_key_name VARCHAR(50) = NULL,
        @grouped_key_name VARCHAR(50) = NULL,
        @grouped_last_update_name VARCHAR(50) = NULL,
        @proto_name VARCHAR(50) = 'NL',
        @columns VARCHAR(256) = '*',
        @layout_type_id INT = 1, -- LASTVALUE
        @last_update_name VARCHAR(50) = 'DATE_UNIX',
        @is_active INT;
        
SET @is_active = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 1
    WHEN 'acceptance' THEN 1
    ELSE 1
END;

SET @view_description = CONCAT('VIEW: ', @view_name, ' FOR ', @item_name);
SET @config_description = CONCAT('VIEW CONFIGURATION: ', @view_name, ' FOR ', @item_name);

-- 2) DETERMINE VIEW ID & CONFIGURATION ID
DECLARE @constrained INT,
        @grouped INT,
        @view_id BIGINT,
        @config_id BIGINT;

SET @constrained = CASE 
    WHEN @constraint_key_name IS NULL THEN 0
    ELSE 1
END;
SET @grouped = CASE 
    WHEN @grouped_key_name IS NULL THEN 0
    ELSE 1
END;

DELETE FROM [DATATINO_PROTO_1].[CONFIGURATIONS]
WHERE [ID] IN (
    SELECT
        configs.[ID]
    FROM [DATATINO_PROTO_1].[VIEWS] views
    INNER JOIN [DATATINO_PROTO_1].[CONFIGURATIONS] AS configs ON views.[ID] = configs.[VIEW_ID]
        AND configs.[NAME] = @item_name
    INNER JOIN [DATATINO_PROTO_1].[PROTOS] AS protos ON protos.[ID] = configs.[PROTO_ID]
        AND protos.[NAME] = @proto_name
);

SELECT 
    @view_id = [ID]
FROM [DATATINO_PROTO_1].[VIEWS]
WHERE ISNULL([CONSTRAINT_VALUE], 'X') = ISNULL(@constraint_value, 'X')
    AND ISNULL([CONSTRAINT_KEY_NAME], 'X') = ISNULL(@constraint_key_name, 'X')
    AND ISNULL([GROUPED_KEY_NAME], 'X') = ISNULL(@grouped_key_name, 'X')
    AND ISNULL([GROUPED_LAST_UPDATE_NAME], 'X') = ISNULL(@grouped_last_update_name, 'X')
    AND [NAME] = @view_name;

SELECT
    @config_id = configs.[ID]
FROM [DATATINO_PROTO_1].[VIEWS] views
INNER JOIN [DATATINO_PROTO_1].[CONFIGURATIONS] AS configs ON views.[ID] = configs.[VIEW_ID]
    AND configs.[NAME] = @item_name
    AND configs.[VIEW_ID] = @view_id
INNER JOIN [DATATINO_PROTO_1].[PROTOS] AS protos ON protos.[ID] = configs.[PROTO_ID]
    AND protos.[NAME] = @proto_name
WHERE views.[NAME] = @view_name;

-- 3) UPSERT PROTO VIEW(S).....
EXECUTE [DATATINO_PROTO_1].[UPSERT_VIEW]
    @id = @view_id,
    @view_name = @view_name,
    @description = @view_description,
    @last_update_name = @last_update_name,
    @constraint_key_name = @constraint_key_name,
    @constraint_value = @constraint_value,
    @grouped_key_name = @grouped_key_name,
    @grouped_last_update_name = @grouped_last_update_name;

-- 4) UPSERT PROTO CONFIGURATION(S).....
EXECUTE [DATATINO_PROTO_1].[UPSERT_CONFIGURATION]
    @id = @config_id,
    @proto_name = @proto_name,
    @description =  @config_description,
    @view_name = @view_name,
    @item_name = @item_name,
    @constrained = @constrained,
    @grouped = @grouped,
    @columns = @columns,
    @mapping = '=LOWER()',
    @layout_type_id = @layout_type_id,
    @active = @is_active,
    @constraint_key_name = @constraint_key_name,
    @constraint_value = @constraint_value,
    @grouped_key_name = @grouped_key_name,
    @grouped_last_update_name = @grouped_last_update_name;
GO